# جلسه ۶ – خط لوله چند مرحله‌ای (برنامه‌ریزی → اجرا → اصلاح)

از توابع مسیریابی برای انتخاب مدل‌ها در هر مرحله استفاده می‌کند، سپس پاسخ نهایی را اصلاح می‌کند.


### توضیح: نصب وابستگی‌ها
نصب `foundry-local-sdk` و `openai` که برای تماس‌های چت مرحله‌ای مورد نیاز هستند. اجرای مجدد آن بدون مشکل است.


# سناریو
خط لوله مدل چندمرحله‌ای که: (1) یک وظیفه را به مراحل مجزا برنامه‌ریزی می‌کند، (2) هر مرحله را با انتخاب مدل مبتنی بر نیت اجرا می‌کند، (3) یک پاسخ نهایی ترکیبی را اصلاح می‌کند. زنجیره‌سازی قابلیت‌های ناهمگن SLM را برای کارایی در عین حفظ کیفیت نشان می‌دهد.


In [13]:
!pip install -q foundry-local-sdk openai

### توضیح: وارد کردن‌های اصلی  
وارد کردن regex برای تشخیص نیت، Foundry Local manager برای پیوست‌های مربوط به هر alias، و OpenAI client برای تکمیل‌های چت.


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### توضیح: کاتالوگ قابلیت‌ها و قوانین
یک کاتالوگ مبتنی بر قابلیت و قوانین regex تعریف می‌کند که برای تطبیق متن مراحل با دسته‌های نیت (کد، خلاصه‌سازی، طبقه‌بندی، عمومی) استفاده می‌شود. مقادیر اولویت پایین‌تر در صورت وجود چندین مدل که از یک نیت پشتیبانی می‌کنند، برنده می‌شوند.


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### توضیح: هدف، انتخاب مدل و کمک‌کننده چت
ارائه می‌دهد:
- `detect_intent` برای طبقه‌بندی مبتنی بر regex.
- `pick_model` برای انتخاب بهترین نام مستعار بر اساس قابلیت + اولویت.
- `chat` یک پوشش راحت که یک کلاینت برای هر نام مستعار ایجاد کرده و یک پاسخ تک‌مرحله‌ای بازمی‌گرداند.


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### توضیح: عملکرد خط لوله چند مرحله‌ای
خط لوله را اجرا می‌کند: برنامه‌ریزی → تجزیه مراحل → اجرای هر مرحله با مسیریابی مبتنی بر قصد → اصلاح خروجی‌های ترکیبی. یک دیکشنری ساختاریافته برای بررسی یا ارزیابی بازمی‌گرداند.


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### توضیح: اجرای مثال وظیفه
کل فرآیند را بر روی یک وظیفه با محوریت بازسازی اجرا می‌کند که نیت‌های ترکیبی (کد + خلاصه‌سازی) را نشان می‌دهد. دیکشنری نتیجه، طرح خام، خروجی‌های هر مرحله با نام‌های مستعار مدل انتخاب‌شده، و پاسخ نهایی ترکیب‌شده را نشان می‌دهد.


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### توضیح: نمایش شیء نتیجه
خروجی ساختاریافته‌ی پایپ‌لاین را برای بررسی سریع یا ارزیابی‌های بعدی (مثلاً اندازه‌گیری کیفیت گام یا اثربخشی بهبود) نشان می‌دهد.



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما هیچ مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
